# Massive Data Processing 
## Spark Practice
Studend: David Sánchez Marín
### 3.- Pair RDDs

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [9]:
# 1. Read the rating data
import os, shutil
import unicodedata
from operator import add

input = sc.textFile("../Datasets/ml-100k/u.data")
users_ori = input.map(lambda l: l.split("\t") )
print "1. Read ratings data"
print ""
print "* Num. users/items: ", users_ori.count()
print ""
print "* User ratings: ", users_ori.first()
users = users_ori.map(lambda u: (int(u[0]), float(u[2]), int(u[1])))
print ""
print "* Numerical user rating: ", users.first()
users_dic = users_ori.map(lambda u: {"user_id": int(u[0]), "item_id":int(u[1]), "rating":float(u[2]), "timestamp":u[3]})
print ""
print "Users conversion: ", users_dic.take(3)

1. Read ratings data

* Num. users/items:  100000

* User ratings:  [u'196', u'242', u'3', u'881250949']

* Numerical user rating:  (196, 3.0, 242)

Users conversion:  [{'item_id': 242, 'rating': 3.0, 'user_id': 196, 'timestamp': u'881250949'}, {'item_id': 302, 'rating': 3.0, 'user_id': 186, 'timestamp': u'891717742'}, {'item_id': 377, 'rating': 1.0, 'user_id': 22, 'timestamp': u'878887116'}]


In [49]:
user_ratings = users.map(lambda u: (u[0], (u[1],u[2])))
print "2. Pair RDD: "
print user_ratings.take(5)

2. Pair RDD: 
[(196, (3.0, 242)), (186, (3.0, 302)), (22, (1.0, 377)), (244, (2.0, 51)), (166, (1.0, 346))]


In [46]:
print "3. Agregate user ratings"
    
user_agg = user_ratings.aggregateByKey((0,0,0),\
        (lambda x, y: (x[0]+y[0],x[1]+1)),\
        (lambda rdd1, rdd2: (rdd1[0]+rdd2[0],rdd1[1]+rdd2[1])))
print user_agg.take(5)

3. Agregate user ratings
[(1, (982.0, 272)), (2, (230.0, 62)), (3, (151.0, 54)), (4, (104.0, 24)), (5, (503.0, 175))]


In [58]:
print "4. User average ratings: "

user_avg = user_agg.map(lambda (x, y): (x, y[0] / y[1]))
print user_avg.take(4)

4. User average ratings: 
[(1, 3.610294117647059), (2, 3.7096774193548385), (3, 2.7962962962962963), (4, 4.333333333333333)]


In [64]:
print "5. Movie number of ratings"
movies = users.map(lambda x: (x[2], x[1]))
movies_rating = movies.reduceByKey(lambda x, y: x + 1)
print movies_rating.take(5)

5. Movies number of ratings
[(1, 455.0), (2, 133.0), (3, 91.0), (4, 213.0), (5, 88.0)]


In [72]:
print "6. High Rating Movies"
movies_up_mean = movies.filter(lambda x: x[1]>3)
print "High rating movies: ", movies_up_mean.distinct().count()
print "Movies high rateg: ", movies_up_mean.take(5)
movies_up_count = movies_up_mean.map(lambda x: (x[0], 1))\
                .reduceByKey(lambda x, y: x+y)
print "Movies num high ratings: ", movies_up_count.take(5)

 6. High Rating Movies
High rating movies:  2556
Movies high rateg:  [(474, 4.0), (465, 5.0), (1014, 5.0), (222, 5.0), (387, 5.0)]
Movies num high ratings:  [(1, 321), (2, 51), (3, 34), (4, 122), (5, 39)]
